# Create card pool database

In [1]:
import polars as pl
import uuid
import random
# import itertools

# 1. mana / advancing / shield / cond / effect card pool

create raw card pool (faction by faction)<br>
build all combinations possible of effects/conditions/mana ...

In [2]:
faction_name = "Twigs"    # choose one faction to build

global_policy = {
    'mana': [1, 2, 3, 4],                 # the mana costs that will be used in the game
    'advancement_range': [-2, 4],         # the advancement min and max that the faction will be allowed to move
}

factions_dict = {
    "Dwarves": {
        "mana_shift": 1,
        "advancement": 0,
        "shield": 1,
        "biomes": ["ocean", "mountains"],
        "logo": r'..\cards_assets\logo_dwarves.png',
        # ---- text for prompt generation ----
        "art_char_age": [' young', '', 'n old'],           # espace important
        "art_char_type": ['', 'dark skin'],
        "art_body": ['', 'fat', 'slim', 'muscular'],    # épineux pour les treants / siamois pour les miaous
        "art_char_sex": ['male', 'female'],
        "art_char_wearing": ["a dark-brown and blue armor"],
        "art_char_accessory": ["boots", "helmet", "gloves", "scarf", "belt", "necklace", "earrings", "glasses", "anklet", "bracelet", "shield", "sword"], # "suspenders", "headband"
        "art_char_global": "A*art_char_age* *art_char_type* *art_body* *art_char_sex* dwarf wearing *art_char_wearing* with *color* shiny *art_char_accessory*",
        "art_animal": "bear",
        "art_ressource": "fruits and vegetables",
        "art_flying": "hot air balloon",
        "art_attack_engine": "trebuchet",
        "art_scene_outdoor": [
            "In a grassy mountainous landscape",
            "In a snowy mountainous landscape",
            "On a cliff overlooking the ocean",
            "In a Dwarven *object_styles* village with stone houses and cobblestone streets",
            "On a pebble beach",
            "At the entrance of a vast dwarven mine, with *object_styles* carts and pickaxes scattered around",
            "On a huge dwarven *object_styles* stone bridge over a deep chasm",
            "In a stone quarry, with dwarves carving massive blocks of rock",
            "At an outdoor dwarven forge, with glowing metal and smoke rising into the air",
            "On a mountain summit with a view of a sprawling dwarven valley"
        ],
        "art_scene_indoor": [
            "In a cozy tavern with wooden beams and a roaring fireplace",
            "In a bustling marketplace filled with colorful stalls",
            "In a *object_styles* blacksmith's forge with tools and weapons hanging on the walls",
            "in a cozy *object_styles* house",
            "In a grand dwarven throne room, with massive stone pillars and golden banners",
            "Inside a magnificent dwarven treasury, overflowing with gold, jewels, and ancient artifacts",
            "Deep within a bustling dwarven mine, with glowing dark-purple crystals lighting the way",
            "In a massive *object_styles* forge, with lava flows and huge bellows",
            "In a detailed workshop, with intricate clockwork devices and glowing blueprints",
            "In a grand dwarven hall, with banners hanging from the high ceilings",
            "Inside a vast dwarven library, with scrolls and stone tablets lining the shelves"
        ],
        "art_neg_prompt": f"",
        # ---- effects / conditions ----
        "conditions": [     # (condition, value associated), value: how much should it give as a bonus
            ("no_condition", 0),
            ("biome_Dwa", 1),
            ("mana_inf_6", 1),
            ("mana_sup_5", 1),
            ("cards_in_hand_inf_4", 1),
            ("cards_in_hand_sup_3", 1),
            ("dist_behind_sup_1", 2),
            ("dist_behind_sup_3", 3),
            ("dist_ahead_sup_1", 1),
            ("dist_ahead_sup_3", 2),
            ("pending", 1),
            ("drop_on_board", 1),
            ("cataclysm", 2),
            ("face_point_right", 1),
            ('face_point_left', 2),
            ("day", 1),
            # ("night", 1),
            # ("temp_sup_9", 1),
            # ("temp_sup_15", 3),
            ("temp_inf_11", 1),
            ("temp_inf_6", 3),
        ],
        "effects": [        # (effect, effect number, value associated [cost!]), value: how much should it cost to get this effect
            ("advancing", [1, 2, 3], [0, -1, -2]),      # shared with all factions
            ("draw", [1, 2], [-2, -3]),                 # shared with all factions
            ("jump", [0], [0]),                          # shared with all factions
            ("wrecking_ball", [0], [-2]),                # shared with all factions
            # ("advancing_oppo", [1, 2], [2, 3]),
            # ("backward", [-1, -2], [2, 3]),
            # ("backward_oppo", [-1], [0]),
            # ("draw_oppo", [1, 2], [2, 3]),
            # ("discard", [-1], [2]),
            # ("discard_oppo", [-1], [-2]),
            ("unstoppable", [0], [-2]),
            ("ramp", [1], [-2]),
            # ("ramp_oppo", [1], [3]),
            # ("taxation", [1], [3]),
            # ("taxation_oppo", [1], [-3]),
            ("grappling_hook", [0], [-2]),
            # ("pet_trap", [0], [-1]),                   # faction Miaous
            ("avalanche", [0], [-2]),                   # faction dwarves
            # ("rooted", [0], [-2]),                     # faction treants - change this with -2 of total advancement?
            # ("copy_effect", [0], [-2]),                # faction mummies
            # ("effect_canceled", [0], [-2]),            # faction demons
            # ("swap_cards", [0], [-1]),                 # faction orcs
        ],
        # please give me a python list (in row) with 50 Dwarves male/female first-names - Gemini
        "male_names": ["Argun", "Balin", "Borin", "Brambur", "Brok", "Dagmar", "Durin", "Dvalin", "Elik", "Fargrim", "Farond", "Finnur", "Gardok", "Ghoran", "Gimlir", "Glorund", "Golgrim", "Gorin", "Grak", "Grimbul", "Grond", "Halvor", "Helgar", "Hrek", "Jarok", "Kaelin", "Karag", "Khazad", "Korgan", "Lorik", "Magni", "Mogran", "Moradin", "Norik", "Odrun", "Orik", "Raknar", "Rhorin", "Rurik", "Sorok", "Tharik", "Thogrim", "Thorin", "Thror", "Urgar", "Vargrim", "Vorlag", "Yarkon", "Yorik", "Zarik"],
        "female_names": ["Alva", "Bara", "Brunhilda", "Dagrun", "Draga", "Dwalina", "Elda", "Faralda", "Fimburla", "Friga", "Greda", "Helga", "Hildir", "Idra", "Ingara", "Karga", "Kharla", "Khazra", "Korla", "Lira", "Lorina", "Magda", "Mirna", "Morwena", "Nira", "Odina", "Olara", "Olma", "Rhina", "Rona", "Sorina", "Thorda", "Thora", "Thurma", "Ursa", "Vala", "Vilda", "Zira", "Zanna", "Branka", "Dorna", "Elda", "Filda", "Grizelda", "Hanna", "Ilsa", "Lurina", "Mardina", "Odila", "Ylva"],
    },
    "Demons": {
        "mana_shift": 0,
        "advancement": 0,
        "shield": -1,
        "biomes": ["ocean", "desert"],
        "logo": r'..\cards_assets\logo_demons.png',
        # ---- text for prompt generation ----
        "art_char_age": [' young', '', 'n old'],           # espace important
        "art_char_type": ['big straight', 'big curvy', "straight", "curvy", 'small straight', 'small curvy'],
        "art_body": ['', 'fat', 'slim', 'muscular'],    # épineux pour les treants / siamois pour les miaous
        "art_char_sex": ['male', 'female'],
        "art_char_wearing": ["dark-purple mage's robe"],
        "art_char_accessory": ["gloves", "scarf", "belt", "necklace", "earrings", "glasses", "anklet", "bracelet", "magic wand", "horn rings", "tail armor", "facial piercings"], # "suspenders", "headband"
        "art_char_global": "A*art_char_age* *art_body* *art_char_sex* tiefling with *art_char_type* *color* horns wearing *art_char_wearing* with *color* shiny *art_char_accessory*",
        "art_animal": "dragon",
        "art_ressource": "purple shards",
        "art_flying": "dragon",
        "art_attack_engine": "mechanical dragon shaped",
        "art_scene_outdoor": [ # please give me a python list with 10 Tiefling (demons) outdoor scenes - Gemini
            "On a volcanic plain with rivers of glowing lava",
            "In a barren hellscape under a blood-red sky",
            "In a corrupted forest where gnarled trees have black leaves",
            "At a ruined fortress with fiendish symbols carved into the stone",
            "Near a river of fire, with black, obsidian rocks",
            "On a windswept mountain pass with ominous demonic statues",
            "In a shadowy necropolis with sinister monuments",
            "In a toxic swamp with bubbling pools and unnatural flora",
            "At an infernal summoning circle drawn on the ground",
            "On a charred battlefield under a sky filled with dark smoke",
            "In a Tiefling *object_styles* village with obsidian houses and cobblestone streets",
            "On a black sand beach with shards of obsidian",
            "At the entrance of a vast infernal dungeon, with *object_styles* torches and chains scattered around",
        ],
        "art_scene_indoor": [
            "In a grand throne room of black obsidian and bone",
            "Inside an infernal forge with lava flowing through channels in the floor",
            "In a shadowy library filled with scrolls of dark magic and cursed tomes",
            "Inside a ceremonial ritual chamber with a glowing summoning circle on the floor",
            "In a Tiefling dwelling with horned skulls and dark tapestries on the walls",
            "Deep within a hellish treasury, with gold and jewels that hum with dark energy",
            "In a bustling Tiefling marketplace, carved from the rock of an ancient volcano",
            "In a sinister torture chamber, with menacing chains and instruments of torment",
            "Inside a clandestine alchemist's lab, filled with bubbling, foul-smelling concoctions",
            "In a dark tavern where the patrons have glowing eyes and drinks are lit with fire"
        ],
        "art_neg_prompt": f"",
        # ---- effects / conditions ----
        "conditions": [     # (condition, value associated), value: how much should it give as a bonus
            ("no_condition", 0),
            ("biome_Dem", 1),
            ("mana_inf_6", 1),
            ("mana_sup_5", 1),
            ("cards_in_hand_inf_4", 1),
            ("cards_in_hand_sup_3", 1),
            ("dist_behind_sup_1", 2),
            ("dist_behind_sup_3", 3),
            ("dist_ahead_sup_1", 1),
            ("dist_ahead_sup_3", 2),
            ("pending", 1),
            ("drop_on_board", 1),
            ("cataclysm", 2),
            ("face_point_right", 1),
            ('face_point_left', 2),
            # ("day", 1),
            ("night", 1),
            # ("temp_sup_9", 1),
            # ("temp_sup_15", 3),
            ("temp_inf_11", 1),
            ("temp_inf_6", 3),
        ],
        "effects": [        # (effect, effect number, value associated [cost!]), value: how much should it cost to get this effect
            ("advancing", [1, 2, 3], [0, -1, -2]),      # shared with all factions
            ("draw", [1, 2], [-2, -3]),                 # shared with all factions
            ("jump", [0], [0]),                         # shared with all factions
            ("wrecking_ball", [0], [-2]),               # shared with all factions
            # ("advancing_oppo", [1, 2], [2, 3]),
            # ("backward", [-1, -2], [2, 3]),
            ("backward_oppo", [-1], [0]),
            # ("draw_oppo", [1, 2], [2, 3]),
            ("discard", [-1], [2]),
            ("discard_oppo", [-1], [-2]),
            # ("unstoppable", [0], [-2]),
            # ("ramp", [1], [-2]),
            # ("ramp_oppo", [1], [3]),
            # ("taxation", [1], [3]),
            ("taxation_oppo", [1], [-3]),
            # ("grappling_hook", [0], [-2]),
            # ("pet_trap", [0], [-1]),                   # faction Miaous
            # ("avalanche", [0], [-2]),                   # faction dwarves
            # ("rooted", [0], [-2]),                     # faction treants - change this with -2 of total advancement?
            # ("copy_effect", [0], [-2]),                # faction mummies
            ("effect_canceled", [0], [-2]),            # faction demons
            # ("swap_cards", [0], [-1]),                 # faction orcs
        ],
        # please give me a python list (in row) with 50 Tiefling (demons) male/female first-names - Gemini
        "male_names": ["Akmenos", "Balasar", "Barakas", "Damakos", "Ekemon", "Iados", "Kairon", "Leucis", "Mehen", "Mordai", "Mortimer", "Pelaios", "Sarx", "Septimus", "Shadus", "Talon", "Thrak", "Valerius", "Xylos", "Zarkas", "Zith", "Phelaia", "Ember", "Seraphina", "Lia", "Ria", "Myllia", "Kala", "Izma", "Nemeia", "Zina", "Valia", "Yana", "Nia", "Mala", "Korinna", "Talia", "Urina", "Vex", "Anakis", "Falx", "Loric", "Rhys", "Silas", "Vrak", "Zadrian", "Zavros", "Zelena", "Nox", "Vespira"],
        "female_names": ["Akela", "Balisa", "Byla", "Damala", "Emira", "Helia", "Irelia", "Kalista", "Korina", "Lelith", "Malyza", "Melora", "Myra", "Nyssa", "Orianna", "Pylia", "Rhianna", "Seraphina", "Sorath", "Thalia", "Zafira", "Zara", "Zylia", "Aziza", "Cyllia", "Damaia", "Elara", "Erelia", "Faenya", "Gwynia", "Hesperia", "Inara", "Jazra", "Kaelia", "Lyrana", "Maelia", "Nirvana", "Orelia", "Phyrra", "Qorra", "Raelia", "Saphira", "Sylvina", "Talia", "Uriana", "Vixena", "Xylona", "Yuna", "Zarina", "Zylia"],
    },
    "Twigs": {
        "mana_shift": 0,
        "advancement": 0,
        "shield": 0,
        "biomes": ["ocean", "jungle"],
        "logo": r'..\cards_assets\logo_twigs.png',
        # ---- text for prompt generation ----
        "art_char_age": [' young', '', 'n old'],           # espace important
        "art_char_type": ['thorny', 'leafy', 'bark-covered', 'covered with ivy', 'covered with moss'],
        "art_body": ['', 'large', 'thin', 'tall', 'small'],    # épineux pour les treants / siamois pour les miaous
        "art_char_sex": ['male', 'female'],
        "art_char_wearing": ["flowers", "small flowers", "big flowers", "faded flowers", "small faded flowers", "big faded flowers"],
        "art_char_accessory": [""], # "suspenders", "headband"
        "art_char_global": "A*art_char_age* *art_body* *art_char_type* *art_char_sex* treant with *color* leaves and *color* *art_char_wearing*",
        "art_animal": "elephant",
        "art_ressource": "bees",
        "art_flying": "massive, harnessed swarm of bees",
        "art_attack_engine": "giant stone golem",
        "art_scene_outdoor": [ # please give me a python list with 10 Tiefling (demons) outdoor scenes - Gemini
            "In a very dense, overgrown jungle",
            "In an ancient, sun-dappled grove with mossy stones",
            "In swampy wetlands with twisted, gnarled trees",
            "In a toxic swamp with bubbling pools and unnatural flora",
            "At a ruined fortress with fiendish symbols carved into the stone",
            "Near a big river",
            "In a Treant *object_styles* village with houses in trees",
            "In a big *object_styles* boat made out of leaves in the middle of a jungle river",
            "In a misty valley where colossal, silent trees stand in the fog",
            "In a dense, overgrown forest floor with glowing mushrooms and soft moss",
            "In a serene, sunlit meadow at the edge of an old-growth forest",
            "In a rain-soaked forest with dripping leaves and glistening bark",
        ],
        "art_scene_indoor": [
            "Inside a massive, hollowed-out tree trunk lit by glowing *color* fungi",
            "In a vast underground grotto with *color* roots hanging from the ceiling",
            "In a cavern where ancient, gnarled roots converge and form natural arches",
            "In a huge chamber deep within the earth with a waterfall and bioluminescent *color* crystals",
            "In the heartwood of a colossal tree, with sunlight filtering through small holes",
            "In an underground crypt for ancient Treants, with roots forming coffins and altars",
            "In a natural, cavernous dwelling with a hearth made of glowing embers and stone",
            "In a labyrinthine tunnel system with Treant roots forming the walls and ceiling",
            "In a grotto containing a massive *color* geode, with roots entwined around its glowing crystals",
            "In a giant wooden temple with intricate carvings and natural light filtering through the canopy"
        ],
        "art_neg_prompt": f"",
        # ---- effects / conditions ----
        "conditions": [     # (condition, value associated), value: how much should it give as a bonus
            ("no_condition", 0),
            ("biome_Twi", 1),
            ("mana_inf_6", 1),
            ("mana_sup_5", 1),
            ("cards_in_hand_inf_4", 1),
            ("cards_in_hand_sup_3", 1),
            ("dist_behind_sup_1", 2),
            ("dist_behind_sup_3", 3),
            ("dist_ahead_sup_1", 1),
            ("dist_ahead_sup_3", 2),
            ("pending", 1),
            ("drop_on_board", 1),
            ("cataclysm", 2),
            ("face_point_right", 1),
            ('face_point_left', 2),
            ("day", 1),
            # ("night", 1),
            ("temp_sup_9", 1),
            ("temp_sup_15", 3),
            # ("temp_inf_11", 1),
            # ("temp_inf_6", 3),
        ],
        "effects": [        # (effect, effect number, value associated [cost!]), value: how much should it cost to get this effect
            ("advancing", [1, 2, 3], [0, -1, -2]),      # shared with all factions
            ("draw", [1, 2], [-2, -3]),                 # shared with all factions
            ("jump", [0], [0]),                         # shared with all factions
            ("wrecking_ball", [0], [-2]),               # shared with all factions
            ("advancing_oppo", [1, 2], [2, 3]),
            # ("backward", [-1, -2], [2, 3]),
            # ("backward_oppo", [-1], [0]),
            ("draw_oppo", [1, 2], [2, 3]),
            # ("discard", [-1], [2]),
            # ("discard_oppo", [-1], [-2]),
            # ("unstoppable", [0], [-2]),
            # ("ramp", [1], [-2]),
            ("ramp_oppo", [1], [3]),
            ("taxation", [1], [3]),
            # ("taxation_oppo", [1], [-3]),
            # ("grappling_hook", [0], [-2]),
            # ("pet_trap", [0], [-1]),                   # faction Miaous
            # ("avalanche", [0], [-2]),                   # faction dwarves
            ("rooted", [0], [-2]),                     # faction treants - change this with -2 of total advancement?
            # ("copy_effect", [0], [-2]),                # faction mummies
            # ("effect_canceled", [0], [-2]),            # faction demons
            # ("swap_cards", [0], [-1]),                 # faction orcs
        ],
        # please give me a python list (in row) with 50 Tiefling (demons) male/female first-names - Gemini
        "male_names": ["Oakheart", "Mossbeard", "Ironwood", "Stonebark", "Gnarledroot", "Elderwood", "Whisperingbough", "Oldgrowth", "Rootstrider", "Bristlecone", "Boulderhelm", "Flintbark", "Grovewood", "Timberjaw", "Bramblethorn", "Cedarfall", "Cypress", "Elm", "Fir", "Aspen", "Willow", "Hemlock", "Juniper", "Larch", "Redwood", "Sycamore", "Thornwood", "Yew", "Ashfang", "Birchbark", "Cragwood", "Dendron", "Fellroot", "Glimmerwood", "Gnarl", "Hollowtrunk", "Kindling", "Leaf-arm", "Lumber", "Mossfoot", "Pinecrush", "Quaking", "Rootfang", "Sapling", "Timberhoof", "Wickerbark", "Windhollow", "Woodshatter", "Yggdrasil", "Zygote"],
        "female_names": ["Willowbloom", "Mossbella", "Irona", "Stoneweaver", "Gnarleana", "Elderwood", "Whisperwind", "Rootlore", "Fernsong", "Bramblelyn", "Aspenleaf", "Cypressa", "Elmyra", "Fira", "Junia", "Larchana", "Redwooda", "Sycamora", "Yeva", "Ashella", "Bircha", "Cragella", "Dendria", "Felldora", "Glissandra", "Gnarlia", "Hollownessa", "Kindlina", "Leaf-armia", "Mossfoot", "Pinecrusha", "Quakena", "Rootfanga", "Saplinga", "Timberah", "Wickera", "Windhollia", "Woodshena", "Yggdrasila", "Acacia", "Briar", "Cedar", "Dahlia", "Embera", "Fawn", "Gemma", "Hazel", "Iridessa", "Juniper", "Laurel"],
    },
    "Miaous": {
        "mana_shift": 0,
        "advancement": 1,
        "shield": 0,
        "biomes": ["jungle", "desert"],
        "logo": r'..\cards_assets\logo_miaous.png',
        # ---- text for prompt generation ----
        "art_char_age": [' young', '', 'n old'],           # espace important
        "art_char_type": ['Persian', 'Maine Coon', 'Siamese', 'British Shorthair', 'Bengal', 'Himalayan', 'Abyssinian', 'Ragdoll' ,'Turkish Angora', 'orange', 'scottish straight'],
        "art_body": ['', 'fat', 'slim'],    # épineux pour les treants / siamois pour les miaous
        "art_char_sex": ['male', 'female'],
        "art_char_wearing": ["a *color* skintight spacesuit"],
        "art_char_accessory": ['suspenders', 'earring', 'helmet', 'gloves', 'belt', 'glasses', 'necklace', 'small jetpack', 'watch'], # "suspenders", "headband"
        "art_char_global": "A*art_char_age* *art_body* tabaxi *art_char_sex* *art_char_type* cat wearing *art_char_wearing* with *color* shiny *art_char_accessory*",
        "art_animal": "dog",
        "art_ressource": "fishes",
        "art_flying": "big jetpack",
        "art_attack_engine": "bulldozer",
        "art_scene_outdoor": [ # please give me a python list with 10 tabaxi cat outdoor background scenes - Gemini
            "In a vibrant jungle with ancient, overgrown *object_styles* ruins",
            "On a sun-scorched savanna with tall, whispering grasses",
            "On a high canyon cliff overlooking a vast, arid desert",
            "In a remote Tabaxi *object_styles* tree village with woven rope bridges",
            "Near a hidden waterfall in a lush, tropical rainforest",
            "On a sun-drenched savanna, with a sense of adventure in the air",
            "In a deep, sun-baked canyon leading to a hidden oasis",
            "In a field of tall bamboo, with light filtering through the stalks",
            "Along the banks of a winding jungle river at dusk",
            "On a mossy forest floor, with a dense canopy blocking the light"
        ],
        "art_scene_indoor": [
            "In a room with sunbeams streaming through high windows, illuminating a soft perch",
            "In an overgrown interior garden with climbing vines and hidden spots for napping and hunting",
            "In a library with towering shelves connected by rope bridges, with books precariously stacked",
            "In a cluttered study with scrolls and trinkets scattered on the floor",
            "In a bright, clean grooming station with tools, combs, and a professional groomer working",
            "On a giant *object_styles* cat bed with plush cushions and dangling toys",
            "In a workshop filled with dangling rope toys and woven spheres",
            "In a massive tree hollow transformed into a home, with sleeping nests tucked into the roots",
            "In an elevated, winding walkway that serves as a common area, with various cat-like lounges and lookouts",
            "In the street in a *object_styles* village"
        ],
        "art_neg_prompt": f"",
        # ---- effects / conditions ----
        "conditions": [     # (condition, value associated), value: how much should it give as a bonus
            ("no_condition", 0),
            ("biome_Mia", 1),
            ("mana_inf_6", 1),
            ("mana_sup_5", 1),
            ("cards_in_hand_inf_4", 1),
            ("cards_in_hand_sup_3", 1),
            ("dist_behind_sup_1", 2),
            ("dist_behind_sup_3", 3),
            ("dist_ahead_sup_1", 1),
            ("dist_ahead_sup_3", 2),
            ("pending", 1),
            ("drop_on_board", 1),
            ("cataclysm", 2),
            ("face_point_right", 1),
            ('face_point_left', 2),
            # ("day", 1),
            ("night", 1),
            ("temp_sup_9", 1),
            ("temp_sup_15", 3),
            # ("temp_inf_11", 1),
            # ("temp_inf_6", 3),
        ],
        "effects": [        # (effect, effect number, value associated [cost!]), value: how much should it cost to get this effect
            ("advancing", [1, 2, 3], [0, -1, -2]),      # shared with all factions
            ("draw", [1, 2], [-3, -4]),                 # shared with all factions
            ("jump", [0], [-1]),                         # shared with all factions
            ("wrecking_ball", [0], [-2]),               # shared with all factions
            # ("advancing_oppo", [1, 2], [2, 3]),
            ("backward", [-1, -2], [2, 3]),
            # ("backward_oppo", [-1], [0]),
            ("draw_oppo", [1, 2], [1, 2]),
            # ("discard", [-1], [2]),
            # ("discard_oppo", [-1], [-2]),
            # ("unstoppable", [0], [-2]),
            ("ramp", [1], [-2]),
            # ("ramp_oppo", [1], [3]),
            ("taxation", [1], [2]),
            # ("taxation_oppo", [1], [-3]),
            # ("grappling_hook", [0], [-2]),
            ("pet_trap", [0], [-2]),                   # faction Miaous
            # ("avalanche", [0], [-2]),                   # faction dwarves
            # ("rooted", [0], [-2]),                     # faction treants - change this with -2 of total advancement?
            # ("copy_effect", [0], [-2]),                # faction mummies
            # ("effect_canceled", [0], [-2]),            # faction demons
            # ("swap_cards", [0], [-1]),                 # faction orcs
        ],
        # please give me a python list (in row) with 50 Tabaxi cat male first-names - Gemini
        "male_names": ["Quick Pounce", "Swift Hunter", "Whispering Wind", "Shadow Walker", "Golden Paw", "Spotted Tail", "River's Flow", "Stone Cutter", "Silent Step", "Sun on the Water", "Roaring Thunder", "Mossy Stone", "Jade Eye", "Burning Ember", "Frost on Leaves", "Cloud Chaser", "Wind Runner", "Claw of the Leopard", "Stalking Tiger", "Hissing Serpent", "Purring Shadow", "Fang", "Claw", "Jasper", "Kael", "Ra'Zul", "Shazad", "Sorin", "Onyx", "Rumble", "Jaguar", "Leap", "Swift", "Nimble", "Dust", "Thorn", "Prowl", "Marrow", "Cinder", "Dreadfang", "Shadowclaw", "Thunderpaw", "Whiskers", "Mister-Cuddles", "Fluffball", "Tigris", "Mister-Mittens", "Whiskers", "Sassy-Cat"],
        "female_names": ["Whispering Willow", "Graceful Leap", "Moon on the Pond", "Silken Paw", "Spotted Lily", "Dew on the Leaf", "Starlight", "River's Echo", "Stone's Whisper", "Sun's Reflection", "Tear of the Moon", "Saffron", "Tigress", "Zola", "Lira", "Shada", "Vira", "Anya", "Nissa", "Furia", "Kalya", "Siana", "Lirael", "Aura", "Nyx", "Ember", "Luna", "Zinnia", "Meera", "Sari", "Thalia", "Zara", "Aethel", "Nera", "Valeria", "Sorina", "Xyla", "Suri", "Elara", "Nari", "Isolde", "Kaia", "Raina", "Selene", "Vesper", "Lina", "Aria", "Jessa", "Kyra", "Lexa"],
    },
    "Orcs": {
        "mana_shift": 1,
        "advancement": 1,
        "shield": -1,
        "biomes": ["mountains", "jungle"],
        "logo": r'..\cards_assets\logo_orcs.png',
        # ---- text for prompt generation ----
        "art_char_age": [' young', '', 'n old'],           # espace important
        "art_char_type": ['', ''],
        "art_body": ['', 'fat', 'slim', ''],    # épineux pour les treants / siamois pour les miaous
        "art_char_sex": ['male', 'female'],
        "art_char_wearing": ["a dark-red torn armor"],
        "art_char_accessory": ["boots", "helmet", "gloves", "belt", "anklet", "bracelet", "axe", "spiked club", "two-handed hammer", "spear", "lightsaber"], # "suspenders", "headband"
        "art_char_global": "A*art_char_age* *art_char_type* *art_body* *art_char_sex* orc wearing *art_char_wearing* with *color* shiny *art_char_accessory*",
        "art_animal": "rhinoceros",
        "art_ressource": "meat",
        "art_flying": "giant bat",
        "art_attack_engine": "Excavator",
        "art_scene_outdoor": [ # please give me a python list with 10 orc outdoor background scenes - Gemini
            "A muddy, churned-up battlefield with broken weapons and armor scattered about",
            "A vast, desolate wasteland with rocks and sparse, dead trees",
            "A crude *object_styles* Orcish war camp, surrounded by spiked palisades and bonfires",
            "The slopes of a smoking volcano, with magma flows visible in the distance",
            "A harsh mountain pass leading up to a fortified *object_styles* Orcish stronghold",
            "The muddy ground near a deep siege trench, littered with scrap metal",
            "A vile, polluted swamp with murky water and twisted, dead foliage",
            "A rickety wooden bridge over a *color* rocky chasm",
            "A rough-hewn clearing where a brutal hunt has just concluded",
            "Under a sickly *color* sky at a polluted riverbank, surrounded by refuse"
        ],
        "art_scene_indoor": [
            "A massive, smoky *object_styles* Orcish Great Hall with rough-hewn timber supports and a central fire pit",
            "A crude War Room with a tattered map spread on a table and spiked weapon racks lining the walls",
            "A filthy *object_styles* dungeon cellblock, with iron bars and chains bolted into damp stone walls",
            "The roaring interior of a primitive Orcish forge, lit by molten metal and black smoke",
            "A crowded, chaotic *object_styles* barracks with crude wooden bunks and scattered, battered armor",
            "A dark Shaman's den, filled with bones, unsettling fetishes, and glowing embers",
            "A treasury vault piled high with mismatched, stolen gold and crude iron chests",
            "A dirty, chaotic *object_styles* kitchen and mess hall, with raw meat hanging from the ceiling hooks",
            "An Orc Warlord's throne room, dominated by a throne made of bone and crude iron",
            "A gruesome *object_styles* butchery chamber with stone slab tables and bloodstains covering the floor"
        ],
        "art_neg_prompt": f"",
        # ---- effects / conditions ----
        "conditions": [     # (condition, value associated), value: how much should it give as a bonus
            ("no_condition", 0),
            ("biome_Orc", 1),
            ("mana_inf_6", 1),
            ("mana_sup_5", 1),
            ("cards_in_hand_inf_4", 1),
            ("cards_in_hand_sup_3", 1),
            ("dist_behind_sup_1", 2),
            ("dist_behind_sup_3", 3),
            ("dist_ahead_sup_1", 1),
            ("dist_ahead_sup_3", 2),
            ("pending", 1),
            ("drop_on_board", 1),
            ("cataclysm", 1),
            ("face_point_right", 1),
            ('face_point_left', 1),
            ("day", 1),
            # ("night", 1),
            # ("temp_sup_9", 1),
            # ("temp_sup_15", 3),
            ("temp_inf_11", 1),
            ("temp_inf_6", 2),
        ],
        "effects": [        # (effect, effect number, value associated [cost!]), value: how much should it cost to get this effect
            ("advancing", [1, 2, 3], [-1, -2, -3]),      # shared with all factions
            ("draw", [1, 2], [-3, -4]),                 # shared with all factions
            ("jump", [0], [-1]),                         # shared with all factions
            ("wrecking_ball", [0], [-2]),               # shared with all factions
            # ("advancing_oppo", [1, 2], [2, 3]),
            # ("backward", [-1, -2], [2, 3]),
            ("backward_oppo", [-1], [-1]),
            # ("draw_oppo", [1, 2], [1, 2]),
            ("discard", [-1], [2]),
            # ("discard_oppo", [-1], [-2]),
            ("unstoppable", [0], [-3]),
            ("ramp", [1], [-2]),
            # ("ramp_oppo", [1], [3]),
            # ("taxation", [1], [2]),
            # ("taxation_oppo", [1], [-3]),
            # ("grappling_hook", [0], [-2]),
            # ("pet_trap", [0], [-2]),                   # faction Miaous
            # ("avalanche", [0], [-2]),                   # faction dwarves
            # ("rooted", [0], [-2]),                     # faction treants - change this with -2 of total advancement?
            # ("copy_effect", [0], [-2]),                # faction mummies
            # ("effect_canceled", [0], [-2]),            # faction demons
            ("swap_cards", [0], [-2]),                   # faction orcs
        ],
        # please give me a python list (in row) with 50 Tabaxi cat male first-names - Gemini
        "male_names": ["Gront", "Krug", "Zog", "Durak", "Urag", "Thrak", "Grog", "Grish", "Ruk", "Skorn", "Vark", "Zush", "Gorg", "Mog", "Borg", "Shag", "Rog", "Korg", "Darg", "Ghor", "Horg", "Jorg", "Vorg", "Warg", "Xor", "Yor", "Zor", "Azog", "Bagar", "Brogg", "Crog", "Druzg", "Farg", "Gloz", "Hazar", "Irok", "Jargal", "Kazgor", "Logash", "Mozg", "Narok", "Ogron", "Pug", "Razg", "Shaka", "Tusk", "Ugrok", "Volg", "Worg", "Yarz"],
        "female_names": ["Grima", "Shagra", "Vorka", "Zora", "Torka", "Urga", "Groka", "Risha", "Vasha", "Zuka", "Baga", "Caza", "Dagra", "Fusha", "Gora", "Hagra", "Jira", "Kalka", "Loga", "Mazoga", "Narza", "Okra", "Puga", "Riza", "Shala", "Tasha", "Umla", "Volka", "Xena", "Yala", "Zara", "Azra", "Branka", "Dagmar", "Frazka", "Ghara", "Haska", "Izolda", "Jaga", "Kasha", "Lugra", "Morza", "Nazgra", "Okala", "Porka", "Ragza", "Shugra", "Togra", "Ursha", "Viza"],
    },
    "Mummies": {
        "mana_shift": 0,
        "advancement": 0,
        "shield": 0,
        "biomes": ["desert", "mountains"],
        "logo": r'..\cards_assets\logo_dwarves.png',
        # ---- text for prompt generation ----
        "art_char_age": ['', '', '', ' cursed', ' ancient'],           # espace important
        "art_char_type": ['', '', '', 'torned', 'wrapped in soft cotton'],  # egyptian for the mummies
        "art_body": ['', '', '', 'fat'],    # épineux pour les treants / siamois pour les miaous
        "art_char_sex": ['', '', '', '*color*'],
        "art_char_wearing": [""],
        "art_char_accessory": ["boots", "headband", "gloves", "scarf", "cufflinks", "necklace", "earrings", "anklet", "bracelet", "finger ring", "brooch"], # "suspenders", "headband"
        "art_char_global": "A*art_char_age* *art_char_type* *art_body* *art_char_sex* mummy with *color* shiny *art_char_accessory*",
        "art_animal": "giant scorpion",
        "art_ressource": "bones",
        "art_flying": "hang gliding scarab beetle",
        "art_attack_engine": "obelisk",
        "art_scene_outdoor": [ # please give me a python list with 10 mummies outdoor background scenes - Gemini
            "In a vast, sun-scorched desert plain with towering sand dunes",
            "In a vast, sun-scorched desert plain with towering mountains",
            "At the exterior of a colossal, decaying *object_styles* pyramid complex at dawn",
            "In a field of toppled columns and ancient, wind-worn *object_styles* statues",
            "At the base of an enormous *object_styles* Sphinx statue, half-buried in sand",
            "Along the dusty, dry banks of a great, ancient desert river",
            "In a golden desert sunset casting long, eerie shadows across the sands",
            "Near an undead caravan marching across the dunes under a baking sun",
            "In a corrupted, magical oasis surrounded by withered palm trees",
            "In a monumental *object_styles* gateway marking the entrance to a desert tomb",
            "In a dramatic scene during a swirling sandstorm, with only vague shapes visible"
        ],
        "art_scene_indoor": [
            "In a opulent Pharaonic Burial *object_styles* Chamber, centered around a grand stone *object_styles* sarcophagus",
            "In a narrow corridor with walls entirely covered in colorful, ancient hieroglyphs",
            "In a booby-trapped chamber with stone floor tiles ready to shift or collapse",
            "In a dusty *object_styles* Treasure Vault with mounds of *object_styles* artifacts",
            "In an Offering Room with a stone *object_styles* altar, smoking incense, and ritualistic bowls",
            "In a claustrophobic, hidden crypt lined with smaller, simple stone coffins",
            "In a colossal, dimly lit hall supported by massive, painted stone *object_styles* columns",
            "In a dark embalming chamber with ancient tools, oils, and faded linens",
            "In a gallery hallway lined with rows of silent, imposing stone *object_styles* statues",
            "In a winding passage deep within a pyramid, lit by flickering torchlight"
        ],
        "art_neg_prompt": f"human female face, human male face",
        # ---- effects / conditions ----
        "conditions": [     # (condition, value associated), value: how much should it give as a bonus
            ("no_condition", 0),
            ("biome_Dwa", 1),
            ("mana_inf_6", 1),
            ("mana_sup_5", 1),
            ("cards_in_hand_inf_4", 1),
            ("cards_in_hand_sup_3", 1),
            ("dist_behind_sup_1", 2),
            ("dist_behind_sup_3", 3),
            ("dist_ahead_sup_1", 1),
            ("dist_ahead_sup_3", 2),
            ("pending", 1),
            ("drop_on_board", 1),
            ("cataclysm", 2),
            ("face_point_right", 1),
            ('face_point_left', 2),
            # ("day", 1),
            ("night", 1),
            ("temp_sup_9", 1),
            ("temp_sup_15", 3),
            # ("temp_inf_11", 1),
            # ("temp_inf_6", 3),
        ],
        "effects": [        # (effect, effect number, value associated [cost!]), value: how much should it cost to get this effect
            ("advancing", [1, 2, 3], [0, -1, -2]),      # shared with all factions
            ("draw", [1, 2], [-2, -3]),                 # shared with all factions
            ("jump", [0], [0]),                          # shared with all factions
            ("wrecking_ball", [0], [-2]),                # shared with all factions
            ("advancing_oppo", [1, 2], [2, 3]),
            # ("backward", [-1, -2], [2, 3]),
            ("backward_oppo", [-1], [0]),
            # ("draw_oppo", [1, 2], [2, 3]),
            # ("discard", [-1], [2]),
            # ("discard_oppo", [-1], [-2]),
            # ("unstoppable", [0], [-2]),
            # ("ramp", [1], [-2]),
            ("ramp_oppo", [1], [3]),
            # ("taxation", [1], [3]),
            # ("taxation_oppo", [1], [-3]),
            ("grappling_hook", [0], [-2]),
            # ("pet_trap", [0], [-1]),                   # faction Miaous
            # ("avalanche", [0], [-2]),                   # faction dwarves
            # ("rooted", [0], [-2]),                     # faction treants - change this with -2 of total advancement?
            ("copy_effect", [0], [-2]),                # faction mummies
            # ("effect_canceled", [0], [-2]),            # faction demons
            # ("swap_cards", [0], [-1]),                 # faction orcs
        ],
        # please give me a python list (in row) with 50 Mummies male / female first-names - Gemini
        "male_names": ["Ahmose", "Amun", "Anubis", "Bakari", "Beb", "Djedefre", "Esmin", "Hori", "Imhotep", "Ipi", "Kamos", "Kenamun", "Khufu", "Khons", "Meren", "Nebi", "Neferkare", "Pasher", "Ptah", "Ramesses", "Senen", "Seti", "Sobek", "Thutmose", "Udja", "Unas", "Woser", "Xerxes", "Yuya", "Zoser", "Akhen", "Ankhesen", "Amenhotep", "Haris", "Intef", "Ka", "Khenti", "Mentuhotep", "Nakht", "Paneb", "Pepi", "Ra", "Sekhem", "Shenu", "Teti", "Userkaf", "Wenamun", "Yuf", "Za", "Djed"],
        "female_names": ["Ankhesenamun", "Nefertiti", "Hatshepsut", "Nefertari", "Meritaten", "Isis", "Nephthys", "Hathor", "Mut", "Bastet", "Sekhmet", "Kiya", "Ahmes", "Satiah", "Meryt", "Henutsen", "Mutnodjmet", "Tiye", "Baketaten", "Henuttawy", "Iset", "Maatkare", "Nebettawy", "Ramose", "Tadukhipa", "Tuya", "Wadjet", "Ankhes", "Ahmose-Nefertari", "Beket", "Dedet", "Esna", "Fenti", "Gita", "Henut", "Ikhernofret", "Itet", "Kasen", "Luyt", "Maia", "Nebet", "Nofret", "Pakhet", "Qetesh", "Renpet", "Seshemu", "Taia", "Tetisheri", "Uadjit", "Ura"],
    },
}

building cardpool one faction by one faction (to avoid rebuild all if small changes)

In [3]:
faction = factions_dict[faction_name]
all_combinations = []
for mana in global_policy["mana"]:
    for cond in faction["conditions"]:
        for effs in faction["effects"]:
            for eff in zip(effs[1], effs[2]):   # we zip the effect values with their associated cost
                real_mana = mana + faction["mana_shift"]
                cond_name = cond[0]
                cond_cost = cond[1]
                eff_name = effs[0]
                eff_number = eff[0]
                eff_cost = eff[1]
                value = cond_cost + eff_cost
                # print(f'mana: {real_mana} | condition: {cond_name}, cond_cost: {cond_cost} | effect: {eff_name}, effect number: {eff_number}, effect_cost: {eff_cost} --> value: {value}')
                all_combinations.append({
                    "faction": faction_name,
                    "mana": real_mana,
                    "advancing": real_mana + value + faction["advancement"],
                    "shield": real_mana + faction["shield"],
                    "condition": cond_name,
                    "effect": eff_name,
                    "effect_number": eff_number,
                    "rare": False,
                    "condeff_value": value
                })
df = pl.DataFrame(all_combinations)
print(len(df))
# df.write_parquet(f"fac_{faction_name}.parquet")

1008


... then filter

In [4]:
# filtered_df = df.filter(df["advancing"].is_in(faction["advancement_range"]))        # filter by advancement range
filtered_df = df.filter(
    ~((df["effect"] == "unstoppable") & (df["advancing"] < 1))     # unstoppable must at least advance by 1
)
filtered_df = filtered_df.filter(
    ~((filtered_df["effect"] == "jump") & (filtered_df["advancing"] < 2))           # jump must at least advance by 2
)

# create unique id
def make_id(row):
    return f"{row['faction'][:3]}{row['mana']}{row['shield']}_{uuid.uuid4().hex[:6]}"

# Suppress the MapWithoutReturnDtypeWarning by specifying return_dtype
filtered_df = filtered_df.with_columns(
    pl.struct(["faction", "mana", "advancing", "shield"]).map_elements(make_id, return_dtype=pl.String).alias("card_id")
)
filtered_df.write_parquet(f"fac_{faction_name}.parquet")
filtered_df

faction,mana,advancing,shield,condition,effect,effect_number,rare,condeff_value,card_id
str,i64,i64,i64,str,str,i64,bool,i64,str
"""Twigs""",1,1,1,"""no_condition""","""advancing""",1,false,0,"""Twi11_2fafb1"""
"""Twigs""",1,0,1,"""no_condition""","""advancing""",2,false,-1,"""Twi11_c359fc"""
"""Twigs""",1,-1,1,"""no_condition""","""advancing""",3,false,-2,"""Twi11_181ea0"""
"""Twigs""",1,-1,1,"""no_condition""","""draw""",1,false,-2,"""Twi11_838008"""
"""Twigs""",1,-2,1,"""no_condition""","""draw""",2,false,-3,"""Twi11_af7a8c"""
…,…,…,…,…,…,…,…,…,…
"""Twigs""",4,9,4,"""temp_sup_15""","""draw_oppo""",1,false,5,"""Twi44_2ae076"""
"""Twigs""",4,10,4,"""temp_sup_15""","""draw_oppo""",2,false,6,"""Twi44_b75c18"""
"""Twigs""",4,10,4,"""temp_sup_15""","""ramp_oppo""",1,false,6,"""Twi44_74ba71"""


## 2. add image Gen AI prompts

in order to get nearly uniques arts we are going to construct a prompt for each card with changing elements in it :

In [ ]:
# Ex.:  A young    fat  dark-skin female  dwarf  wearing brown and blue clothes with shiny red belt, jumping very high in the sky leaving a shiny light trail [with hundreds of glowing pink butterflies around]
# Core:                                                                               {char_global},                                                 {action} [with                              {scene_detail}] (1/100 chance to appear)

prompt to Gemini: what could we use as illustration on a card that make the player go backward, in a TCG where the goal is to advance to a certain number. Gives me 10 recommendations  please

In [5]:
col_type = ['', 'dark-', 'light-']
colors = ["red", "blue", "green", "yellow", "orange", "brown", "purple", "black", "white"]

object_styles = ['', 'futuristic', 'rustic', 'steampunk', 'medieval', 'modern', 'arcane', 'cyberpunk', 'gothic', 'celestial', 'tech-noir', 'tribal',
                 'post-apocalyptic', 'dieselpunk', 'victorian', 'baroque', 'holographic', 'crystalline', 'demonic']

art_effects = {
    # ----- due to effects -----
    "advancing": [          # common to all factions
        'running at high speed leaving on a *color* trail of dust behind. *scene_outdoor*',
        'cycling on a *color* *object_styles* bike at high speed leaving a trail of shimmering energy behind. *scene_outdoor*',
        'riding on a *color* *fact_animal* at high speed leaving a trail of shimmering energy behind. *scene_outdoor*',
        'holding up a glowing *color* map. *scene_indoor*',
        'navigating a river on a small *color* *object_styles* raft, effortlessly avoiding rocks and rapids as they are carried swiftly by the current'
    ],
    "draw": [                 # common to all factions
        'loading a bag of *fact_ressource* into its shed. *scene_outdoor*',
        'collecting *fact_ressource* into a basket. *scene_outdoor*',
        'gazes into a swirling, magical *color* pool. The reflection in the water isn\'t of them, but of several *fact_ressource* rising from the depths',
        'looking at the sky where *fact_ressource* are falling. *scene_outdoor*',
        'unrolling a new, intricately drawn map on a table. As the map unfolds, new roads, landmarks, and undiscovered locations appear on it. *scene_indoor*'
    ],
    "jump": [                 # common to all factions
        'jumping very high in the sky leaving a shiny light trail. *scene_outdoor*',
        'doing a backflip leaving a trail of sparkles. *scene_outdoor*',
        'is launched from a fantastical *object_styles* catapult, sailing through the air. *scene_outdoor*',
        'soaring through the air on a *color* *object_styles* *fact_fly*',
        'is wearing *object_styles* boots with magical springs on the bottom. The boots have just compressed and are now launching the character high into the air. *scene_outdoor*'
    ],
    "wrecking_ball": [        # common to all factions
        'operating a *color* *object_styles* *art_attack_engine* while saying hello to the camera in a wide shot. *scene_outdoor*',
        'using a *color* slingshot on a *object_styles* house. *scene_outdoor*',
        'is blowing on a house literally made of cards, which is elegantly and deliberately being fanned out and scattered. *scene_outdoor*',
        'stands triumphant with a massive, wrecking ball swinging in the background. The ball is a sphere of pure, destructive energy. *scene_outdoor*',
        'is performing a ritual with mystical energy swirling around the house. The house is not being physically destroyed, but is instead being pulled into a different dimension or realm'
    ],
    "advancing_oppo": [
        'pointing the way forward. *scene_outdoor*',
        'blowing on a small toy ship in a bathtub',
        'leaning on a signpost that reads "Finish Line". *scene_outdoor*',
    ],
    "backward": [
        'falling from a roof, with a surprised look. *scene_outdoor*',
        'looking at a *object_styles* path that has crumbled away. *scene_outdoor*',
        'is struggling against a powerful, magical wind that is blowing directly against them. They are bent over, fighting just to stay in place. *scene_outdoor*'
    ],
    "backward_oppo": [
        'violently pushing back a *color* humanoid silhouette which dissipates into shadows surrounded by ethereal dark light. *scene_outdoor*',
        'casting a spell of a shadowy tentacle that pulls a black figure back. *scene_outdoor*',
        'looking at a *object_styles* bridge dissolving into thin air. *scene_outdoor*',
    ],
    "draw_oppo": [
        'loading a big bag of *fact_ressource* into a cart pulled by a *fact_animal*',
        'sitting on the floor, wrapping a parcel with a *color* glowing paper. *scene_indoor*',
        'Facing the camera, with hands forming a heart shape. *scene_indoor*'
    ],
    "discard": [
        'carrying a *object_styles* backpack filled with *fact_ressource*. A large magic *color* hand comes to take the *fact_ressource* away',
        'throwing a *fact_ressource* away. *scene_indoor*',
        'sitting on the floor, looking sadly at a *fact_ressource* being taken away by an invisible glowing force. *scene_indoor*'
    ],
    "discard_oppo": [
        'crouching low like a thief breaking into a *object_styles* house. *scene_outdoor*',
        'casting a spell on a small *color* *object_styles* voodoo doll. *scene_indoor*',
        'crouching low stealing an object from a backpack. *scene_indoor*',
    ],
    "unstoppable": [
        'made of living stone and infused with ancient power, stands firm against the wind. Highlighting its unstoppable and immune nature. Epic fantasy, monumental scale, glowing magical defenses'
    ],
    "ramp": [
        'looking at a powerful geyser of magical glowing dark-purple droplets that erupts from the ground. The energy is shown flowing directly into the player\'s hands symbolizing a sudden and massive surge of power. *scene_outdoor*',
        'pouring a few drops of a glowing dark-purple liquid into a cauldron. The brew inside immediately bubbles and overflows, multiplying into several smaller vials, representing a powerful and efficient duplication of resources. *scene_indoor*',
        'looking at the sky where big glowing dark-purple droplets are falling from the clouds. *scene_outdoor*'
    ],
    "ramp_oppo": [
        'giving a vial full of dark-purple glowing liquid to a *color* shadowy humanoid silhouette surrounded by ethereal dark light. *scene_indoor*',
        'loading a big transparent container of dark-purple glowing liquid into a cart pulled by a *fact_animal*. *scene_outdoor*',
    ],
    "taxation": [
        'looking at a dark-purple glowing liquid falling from his hands. *scene_indoor*'
    ],
    "taxation_oppo": [
        'crouching, placing C4 on a cauldron of dark-purple glowing liquid'
    ],
    "grappling_hook": [
        'riding a *color* *fact_animal* throwing a long *color* magical *object_styles* grappling hook'
    ],

    # ----- faction specific -----
    "pet_trap": [               # Miaous
        'A super cute *color* kitten lying on its back. *scene_indoor*'
    ],
    "avalanche": [              # dwarves
        'casting a *color* spell that triggers a massive avalanche that buries a small village'
    ],
    "rooted": [                 # Treants
        'Rooting itself firmly in the ground with large *color* roots'
    ],
    "copy_effect": [            # mummies
        'facing a large, ornate *color* *object_styles* mirror'
    ],
    "effect_canceled": [        # demons
        'facing the camera the arms in the shape of an X surrounded by ethereal *color* light'
    ],
    "swap_cards": [             # orcs
        'dancing with another orc. *scene_indoor*',
        'playing a game of cards with another orc in a. *scene_indoor*'
    ],

    # ----- due to value -----
    "null_advancement": [
        'Sipping a *color* cocktail in a crowded *object_styles* bar',
        'Sipping a *color* cocktail in a hammock. *scene_outdoor*'
    ],
    "+3_condeff_value": [
        'sitting on a *color* *object_styles* rocket as it blasts off for the moon',
    ],

    # ----- due to conditions -----
    "night": [
        'in a complete dark night',
    ],
    "hot": [
        'drinking cold water from a *object_styles* container while the sun beats really hard. *scene_outdoor*',
        'Exhausted, shielding their eyes with their hand from the scorching sun. *scene_outdoor*'
    ],
    "cold": [
        'Having a snowball fight while it\'s snowing heavily. *scene_outdoor*',
        'sitting by a campfire, warming their hands while it\'s snowing heavily. *scene_outdoor*',
        'Frozen in place, and covered in ice. *scene_indoor*'
    ],
}


add prompts to DB with combined text

In [6]:
# Load card pool for one faction from parquet
df = pl.read_parquet(f"fac_{faction_name}.parquet")

def build_prompt_old(row):
    age = random.choice(ages)
    # job = random.choice(jobs)
    sex = random.choice(factions_dict[row['faction']]['art_char'])
    char = row['faction'][:-1].lower()
    fact_wearing = factions_dict[row['faction']]['art_wearing']
    rare_detail = rare_stomp_layer if random.randint(1, 100) < 3 else ''  # 2 images on 100
    effect_key = row['effect']
    # Use the first matching effect description, fallback to effect as text if not found
    effect_desc = random.choice(art_effects.get(effect_key, [effect_key]))
    accessory = random.choice(accessories)

    biome1 = factions_dict[row['faction']]['biomes'][0]
    biome2 = factions_dict[row['faction']]['biomes'][1]
    night = ''
    if row['condition'] == 'night':
        night = 'in a complete dark night'
    art_style = f"{night}\nBackground: {biome1} and {biome2}\ncozy soft pastel colors, gentle lighting, digital painting, delicate details, realistic, high quality"

    negative_prompt = factions_dict[row['faction']]['art_neg_prompt']

    return (f"A{age} {sex} {char} wearing {fact_wearing} with only one golden shiny {accessory}, {effect_desc} {rare_detail}.{art_style}", negative_prompt)
    # return (f"A{age} {job} {sex} {char} wearing {fact_wearing} with golden shiny {accessory}, {effect_desc} {rare_detail}.{art_style}", negative_prompt)

def build_char_global(char_global, rare):
    age = random.choice(factions_dict[faction_name]['art_char_age'])
    type = random.choice(factions_dict[faction_name]['art_char_type'])
    body = random.choice(factions_dict[faction_name]['art_body'])
    sex = random.choice(factions_dict[faction_name]['art_char_sex'])
    wearing = random.choice(factions_dict[faction_name]['art_char_wearing'])
    accessory = random.choice(factions_dict[faction_name]['art_char_accessory'])
    
    char_global = char_global.replace('*art_char_age*', age)
    char_global = char_global.replace('*art_char_type*', type)
    char_global = char_global.replace('*art_body*', body)
    char_global = char_global.replace('*art_char_sex*', sex)
    char_global = char_global.replace('*art_char_wearing*', wearing)
    char_global = char_global.replace('*art_char_accessory*', accessory)

    if rare:
        if 'mummy' in char_global:
            char_global = char_global.replace('mummy', '*color* mummy')
        char_global = char_global.replace('*color*', 'pink')
    else:
        for i in range(10):
            char_global = char_global.replace('*color*', random.choice(col_type) + random.choice(colors), 1)  # only replace the first occurence of color

    return char_global

def build_action(effect, rare, extra=False):
    if extra:
        effect_txt = effect
    else:
        effect_txt = random.choice(art_effects[effect])

    object_style = random.choice(object_styles)
    animal = factions_dict[faction_name]['art_animal']
    ressource = factions_dict[faction_name]['art_ressource']
    flying_object = factions_dict[faction_name]['art_flying']
    attack_engine = factions_dict[faction_name]['art_attack_engine']
    scene_outdoor = random.choice(factions_dict[faction_name]['art_scene_outdoor'])
    scene_indoor = random.choice(factions_dict[faction_name]['art_scene_indoor'])

    if rare:
        effect_txt = effect_txt.replace('*color*', 'pink')
    else:
        for i in range(10):
            effect_txt = effect_txt.replace('*color*', random.choice(col_type) + random.choice(colors), 1)  # only replace the first occurence of color
    
    effect_txt = effect_txt.replace('*fact_animal*', animal)
    effect_txt = effect_txt.replace('*fact_ressource*', ressource)
    effect_txt = effect_txt.replace('*fact_fly*', flying_object)
    effect_txt = effect_txt.replace('*art_attack_engine*', attack_engine)
    effect_txt = effect_txt.replace('*scene_outdoor*', scene_outdoor)
    effect_txt = effect_txt.replace('*scene_indoor*', scene_indoor)
    effect_txt = effect_txt.replace('*object_styles*', object_style)

    return effect_txt

def build_prompt(row):
    rare = True if random.randint(1, 100) <= 1 else False  # True with 1 chances out of 100
    if rare:
        rare_txt = 'with hundreds of glowing pink butterflies everywhere'
    else:
        rare_txt = ''

    if row['condition'] == 'night':
        night = random.choice(art_effects['night'])
    else:
        night = ''

    char_global = factions_dict[faction_name]['art_char_global']
    char_global = build_char_global(char_global, rare)
    
    if (row['condition'] == 'temp_sup_9') or (row['condition'] == 'temp_sup_15'):
        action = build_action(random.choice(art_effects['hot']), rare, extra=True)
    elif (row['condition'] == 'temp_inf_11') or (row['condition'] == 'temp_inf_6'):
        action = build_action(random.choice(art_effects['cold']), rare, extra=True)
    else:
        action = build_action(row['effect'], rare)
    
    art_style = f"\ncozy soft pastel colors, gentle lighting, digital painting, delicate details, realistic, high quality"

    prompt = f"{char_global}, {action}, {rare_txt}, {night}{art_style}"

    negative_prompt = factions_dict[faction_name]['art_neg_prompt']

    return prompt, negative_prompt, rare

# Store only the prompt text (not tuple) to avoid Object dtype issues with parquet
df = df.with_columns(
    pl.struct(df.columns).map_elements(lambda row: build_prompt(row)[0], return_dtype=pl.String).alias("prompt"),
    pl.struct(df.columns).map_elements(lambda row: build_prompt(row)[1], return_dtype=pl.String).alias("negative_prompt"),
)
df = df.with_columns( # TO BE TESTED !!!!
    pl.when(pl.col("prompt").str.contains("pink"))
      .then(True)
      .otherwise(pl.col("rare"))
      .alias("rare")
)
df.write_parquet(f"fac_{faction_name}.parquet")
df

faction,mana,advancing,shield,condition,effect,effect_number,rare,condeff_value,card_id,prompt,negative_prompt
str,i64,i64,i64,str,str,i64,bool,i64,str,str,str
"""Twigs""",1,1,1,"""no_condition""","""advancing""",1,false,0,"""Twi11_2fafb1""","""An old small bark-covered male…",""""""
"""Twigs""",1,0,1,"""no_condition""","""advancing""",2,false,-1,"""Twi11_c359fc""","""An old large bark-covered fema…",""""""
"""Twigs""",1,-1,1,"""no_condition""","""advancing""",3,false,-2,"""Twi11_181ea0""","""A thin covered with moss male …",""""""
"""Twigs""",1,-1,1,"""no_condition""","""draw""",1,false,-2,"""Twi11_838008""","""A small covered with moss male…",""""""
"""Twigs""",1,-2,1,"""no_condition""","""draw""",2,false,-3,"""Twi11_af7a8c""","""A young tall thorny male trean…",""""""
…,…,…,…,…,…,…,…,…,…,…,…
"""Twigs""",4,9,4,"""temp_sup_15""","""draw_oppo""",1,false,5,"""Twi44_2ae076""","""A young covered with ivy male…",""""""
"""Twigs""",4,10,4,"""temp_sup_15""","""draw_oppo""",2,false,6,"""Twi44_b75c18""","""A thin bark-covered female tre…",""""""
"""Twigs""",4,10,4,"""temp_sup_15""","""ramp_oppo""",1,false,6,"""Twi44_74ba71""","""An old thin covered with moss …",""""""


## 3. add card name

In [7]:
df = pl.read_parquet(f"fac_{faction_name}.parquet")

In [8]:
def build_name(row):
    first_name = random.choice(factions_dict[row['faction']]["male_names"])
    if "female" in row['prompt']:
        first_name = random.choice(factions_dict[row['faction']]["female_names"])  
    last_name = random.choice(factions_dict[row['faction']]["male_names"] + factions_dict[row['faction']]["female_names"])
    return f"{first_name} {last_name}"

df = df.with_columns(
    pl.struct(df.columns).map_elements(lambda row: build_name(row), return_dtype=pl.String).alias("name"),
)
df.write_parquet(f"fac_{faction_name}.parquet")
df

faction,mana,advancing,shield,condition,effect,effect_number,rare,condeff_value,card_id,prompt,negative_prompt,name
str,i64,i64,i64,str,str,i64,bool,i64,str,str,str,str
"""Twigs""",1,1,1,"""no_condition""","""advancing""",1,false,0,"""Twi11_2fafb1""","""An old small bark-covered male…","""""","""Zygote Pinecrush"""
"""Twigs""",1,0,1,"""no_condition""","""advancing""",2,false,-1,"""Twi11_c359fc""","""An old large bark-covered fema…","""""","""Hazel Mossbeard"""
"""Twigs""",1,-1,1,"""no_condition""","""advancing""",3,false,-2,"""Twi11_181ea0""","""A thin covered with moss male …","""""","""Dendron Leaf-armia"""
"""Twigs""",1,-1,1,"""no_condition""","""draw""",1,false,-2,"""Twi11_838008""","""A small covered with moss male…","""""","""Glimmerwood Elderwood"""
"""Twigs""",1,-2,1,"""no_condition""","""draw""",2,false,-3,"""Twi11_af7a8c""","""A young tall thorny male trean…","""""","""Kindling Sycamore"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Twigs""",4,9,4,"""temp_sup_15""","""draw_oppo""",1,false,5,"""Twi44_2ae076""","""A young covered with ivy male…","""""","""Wickerbark Windhollia"""
"""Twigs""",4,10,4,"""temp_sup_15""","""draw_oppo""",2,false,6,"""Twi44_b75c18""","""A thin bark-covered female tre…","""""","""Elderwood Sapling"""
"""Twigs""",4,10,4,"""temp_sup_15""","""ramp_oppo""",1,false,6,"""Twi44_74ba71""","""An old thin covered with moss …","""""","""Glimmerwood Boulderhelm"""


In [ ]:
# region titles by effects
# if row['effect'] == '+1 p':
#     title = "the swift"
# elif row['effect'] == '+2 p':
#     title = "the even swifter"
# elif row['effect'] == '+1 oppo':
#     title = "the selfless"
# elif row['effect'] == '+2 oppo':
#     title = "the even selfless"
# elif row['effect'] == '-1 p':
#     title = "the stumbling"
# elif row['effect'] == '-2 p':
#     title = "the even stumbling"
# elif row['effect'] == '-1 oppo':
#     title = "the annoying"
# elif row['effect'] == '-2 oppo':
#     title = "the even annoying"
# elif row['effect'] == 'draw 1 p':
#     title = "the insightful"
# elif row['effect'] == 'draw 2 p':
#     title = "the even insightful"
# elif row['effect'] == 'draw 1 oppo':
#     title = "the feeder"
# elif row['effect'] == 'draw 2 oppo':
#     title = "the even feeder"
# elif row['effect'] == 'discard 1 p':
#     title = "the sacrificer"
# elif row['effect'] == 'discard 1 oppo':
#     title = "the saboteur"
# elif row['effect'] == 'unstoppable':
#     title = "the rock"
# elif row['effect'] == 'jump':
#     title = "the kangaroo"
# elif row['effect'] == 'scry':
#     title = "the soothsayer"
# elif row['effect'] == 'wrecking ball':
#     title = "the breaker"
# elif row['effect'] == 'zulu honey trap':
#     title = "the trapper"
# elif row['effect'] == 'grappling hook':
#     title = "the leech"
# elif row['effect'] == 'avalanche':
#     title = "the snowstorm"
# elif row['effect'] == 'both advance':
#     title = "the generous"
# elif row['effect'] == 'nobody moves':
#     title = "the preventing"
# elif row['effect'] == 'effect canceled':
#     title = "the nullifying"
# endregion

## 4. Gathering all faction_DB into one (cardpool)

In [16]:
FACTIONS = list(factions_dict.keys())
FACTIONS

cardpool = None

for faction_name in FACTIONS:
    print(f"Generating cardpool for faction: {faction_name}")
    df = pl.read_parquet(f'fac_{faction_name}.parquet')
    print(df.head(5))
    if cardpool is None:
        cardpool = df
    else:
        cardpool = pl.concat([cardpool, df])

cardpool

# # Save the concatenated DataFrame
cardpool.write_parquet("cardpool.parquet")
cardpool

Generating cardpool for faction: Dwarves
shape: (5, 13)
┌─────────┬──────┬───────────┬────────┬───┬──────────────┬─────────────┬─────────────┬─────────────┐
│ faction ┆ mana ┆ advancing ┆ shield ┆ … ┆ card_id      ┆ prompt      ┆ negative_pr ┆ name        │
│ ---     ┆ ---  ┆ ---       ┆ ---    ┆   ┆ ---          ┆ ---         ┆ ompt        ┆ ---         │
│ str     ┆ i64  ┆ i64       ┆ i64    ┆   ┆ str          ┆ str         ┆ ---         ┆ str         │
│         ┆      ┆           ┆        ┆   ┆              ┆             ┆ str         ┆             │
╞═════════╪══════╪═══════════╪════════╪═══╪══════════════╪═════════════╪═════════════╪═════════════╡
│ Dwarves ┆ 2    ┆ 2         ┆ 3      ┆ … ┆ Dwa23_778c0f ┆ A young     ┆             ┆ Ingara      │
│         ┆      ┆           ┆        ┆   ┆              ┆ fat female  ┆             ┆ Raknar      │
│         ┆      ┆           ┆        ┆   ┆              ┆ dwarf wear… ┆             ┆             │
│ Dwarves ┆ 2    ┆ 1         ┆ 3   

faction,mana,advancing,shield,condition,effect,effect_number,rare,condeff_value,card_id,prompt,negative_prompt,name
str,i64,i64,i64,str,str,i64,bool,i64,str,str,str,str
"""Dwarves""",2,2,3,"""no_condition""","""advancing""",1,false,0,"""Dwa23_778c0f""","""A young fat female dwarf wear…","""""","""Ingara Raknar"""
"""Dwarves""",2,1,3,"""no_condition""","""advancing""",2,false,-1,"""Dwa23_4d6733""","""A muscular female dwarf wear…","""""","""Ilsa Dorna"""
"""Dwarves""",2,0,3,"""no_condition""","""advancing""",3,false,-2,"""Dwa23_4e17f7""","""An old dark skin female dwarf…","""""","""Ursa Elik"""
"""Dwarves""",2,0,3,"""no_condition""","""draw""",1,false,-2,"""Dwa23_54db40""","""A young female dwarf wearing…","""""","""Greda Filda"""
"""Dwarves""",2,-1,3,"""no_condition""","""draw""",2,false,-3,"""Dwa23_4c63a4""","""An old slim female dwarf wear…","""""","""Lurina Sorok"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Mummies""",4,10,4,"""temp_sup_15""","""advancing_oppo""",2,false,6,"""Mum44_29380b""","""A cursed red mummy with oran…","""human female face, human male …","""Za Uadjit"""
"""Mummies""",4,7,4,"""temp_sup_15""","""backward_oppo""",-1,false,3,"""Mum44_fa57c0""","""A fat dark-white mummy with b…","""human female face, human male …","""Kamos Henut"""
"""Mummies""",4,10,4,"""temp_sup_15""","""ramp_oppo""",1,false,6,"""Mum44_896c46""","""A torned fat mummy with light…","""human female face, human male …","""Esmin Satiah"""


## 5. Card pool reduction

make all faction have 250 cards max

In [ ]:
# Uniformly sample up to 250 cards per faction
def sample_per_faction(df, n=250, seed=42):
    sampled = (
        df
        .group_by("faction")
        .map_groups(lambda group: group.sample(n=min(n, group.height), seed=seed))
    )
    return sampled

filtered_df = sample_per_faction(filtered_df, n=250)
filtered_df.write_parquet("card_pool.parquet")
filtered_df

## Card pool analysis

In [ ]:
# Group by 'faction' and aggregate
print(f'number of total cards: {len(filtered_df)}')
result = filtered_df.group_by("faction").agg([
    pl.len().alias("num_cards"),
    pl.mean("mana").alias("avg_mana"),
    pl.mean("advancing").alias("avg_advancing"),
    pl.mean("shield").alias("avg_shield"),
    pl.n_unique("condition").alias("unique_conditions"),
    pl.n_unique("effect").alias("unique_effects")
])

print(result)

# Calculate and print the ratio of avg_advancing to avg_mana for each faction
ratio = result["avg_mana"] / result["avg_advancing"] 
# print(f'advancing ratio dwarves: {ratio[0]:.2f}\nadvancing ratio demons: {ratio[1]:.2f}')

# print the count of how many cards has "with hundreds of glowing pink butterflies everywhere" in their prompt
# butterfly_count = filtered_df.filter(pl.col("prompt").str.contains("with hundreds of glowing pink butterflies everywhere")).height
# print(f'\nNumber of cards with butterfly effect: {butterfly_count}')


In [ ]:
# Group by 'faction' and 'effect' and aggregate
result_by_effect = filtered_df.group_by(["faction", "effect"]).agg([
    pl.len().alias("num_cards"),
    pl.mean("mana").alias("avg_mana"),
    pl.mean("advancing").alias("avg_advancing"),
    pl.mean("shield").alias("avg_shield"),
    pl.n_unique("condition").alias("unique_conditions")
])

result_by_effect

## Others

### restore Mummies parquet file from cardool 

In [15]:
df = pl.read_parquet("cardpool.parquet")
df_mummies = df.filter(pl.col("faction") == "Mummies")

df_mummies.write_parquet(f"fac_{faction_name}.parquet")

### value modif

In [ ]:
faction_name = "Dwarves"
df = pl.read_parquet(f"fac_{faction_name}.parquet")

df = df.with_columns(
    pl.col("condition").replace("Temp < 11", "temp_inf_11").alias("condition")
)
df = df.with_columns(
    pl.col("condition").replace("Temp < 6", "temp_inf_6").alias("condition")
)
df = df.with_columns(
    pl.col("condition").replace("Biome_Dwa", "biome_Dwa").alias("condition")
)

df

In [ ]:
filtered_df

In [ ]:
filtered_df = filtered_df.with_columns( # TO BE TESTED !!!!
    pl.when(pl.col("prompt").str.contains("pink"))
      .then(True)
      .otherwise(pl.col("rare"))
      .alias("rare")
)

In [ ]:
df.write_parquet(f"fac_{faction_name}.parquet")